## Testing calls to model server

#### Setup for Seldom MLflow custom model server

In [ ]:
# Config for diabetes-model-server
%env MODEL_SERVER_URL=http://localhost:7000
%env MODEL_NAME=mlflow-model



In [ ]:
%%bash

docker-compose up -d diabetes-model-server


### Setup for Seldom mlserver

In [ ]:
# Config for mlserver (Seldon.IO container)
%env MODEL_SERVER_URL=http://localhost:8080
%env MODEL_NAME=diabetes-model

In [ ]:
%%bash

docker-compose up -d mlserver


### Calls to server

In [65]:
%%bash

curl -X POST -s "${MODEL_SERVER_URL}/v2/models/${MODEL_NAME}/infer" \
     -H "Content-Type: application/json" \
     -d '{
           "inputs": [
             {
               "name": "input-0",
               "shape": [2, 10],
               "datatype": "FP64",
               "data": [
                    [0.038076, 0.050680, 0.061696, 0.021872, -0.044223, -0.034821, -0.043401, -0.002592, 0.019907, -0.017646],
                    [0.041708, 0.059182, 0.063738, 0.022681, -0.042640, -0.034450, -0.042857, -0.002639, 0.020058, -0.017646]
               ]
             }
           ]
         }' | jq


{
  "model_name": "diabetes-model",
  "id": "054614cf-f1fb-4691-b44d-c0eeaf43d292",
  "parameters": {
    "content_type": "np"
  },
  "outputs": [
    {
      "name": "output-1",
      "shape": [
        2,
        1
      ],
      "datatype": "FP64",
      "parameters": {
        "content_type": "np"
      },
      "data": [
        187.53314700070217,
        186.97810774171552
      ]
    }
  ]
}


In [66]:
%%bash
curl -X GET -s "${MODEL_SERVER_URL}/v2/models/${MODEL_NAME}" | jq


{
  "name": "diabetes-model",
  "versions": [],
  "platform": "",
  "inputs": [
    {
      "name": "input-0",
      "datatype": "FP64",
      "shape": [
        -1,
        10
      ],
      "parameters": {
        "content_type": "np"
      }
    }
  ],
  "outputs": [
    {
      "name": "output-0",
      "datatype": "FP64",
      "shape": [
        -1
      ],
      "parameters": {
        "content_type": "np"
      }
    }
  ],
  "parameters": {
    "content_type": "np"
  }
}


In [67]:
%%bash
curl -s -X POST ${MODEL_SERVER_URL}/invocations \
 -H "Content-Type: application/json" \
  -d '{
        "dataframe_split": {
            "columns": ["age", "sex", "bmi", "bp", "s1", "s2", "s3", "s4", "s5", "s6"],
            "data": [
                [0.038076, 0.050680, 0.061696, 0.021872, -0.044223, -0.034821, -0.043401, -0.002592, 0.019907, -0.017646],
                [-0.001882, 0.352598, -0.170647, 0.190409, 0.273711, -0.024980, -0.091299, 0.042257, -0.049390, -0.092804]
            ]
        }
      }' \
  | jq



{
  "predictions": [
    187.53314700070217,
    93.96267376859655
  ]
}


##### Force model unload/load with curl

In [68]:
%%bash

curl -s  -X POST "${MODEL_SERVER_URL}/v2/repository/models/${MODEL_NAME}/unload"


In [69]:
%%bash

curl -s -X POST "${MODEL_SERVER_URL}/v2/repository/models/${MODEL_NAME}/load" | jq


Check model status with repository index call

In [70]:
%%bash

curl -s -X POST "${MODEL_SERVER_URL}/v2/repository/index" \
  -H "Content-Type: application/json" \
  -d "{}" | jq

[
  {
    "name": "diabetes-model",
    "state": "READY",
    "reason": ""
  }
]


#### Using Custom docker container model-server

In [71]:
%%bash

curl -s -X POST "${MODEL_SERVER_URL}/invocations" \
 -H "Content-Type: application/json" \
  -d '{
        "dataframe_split": {
            "columns": ["age", "sex", "bmi", "bp", "s1", "s2", "s3", "s4", "s5", "s6"],
            "data": [
                [0.038076, 0.050680, 0.061696, 0.021872, -0.044223, -0.034821, -0.043401, -0.002592, 0.019907, -0.017646],
                [-0.001882, -0.044642, -0.118358, -0.034689, -0.045946, -0.034157, -0.072402, 0.042324, -0.009637, 0.034509]
            ]
        }
      }' \
  | jq


{
  "predictions": [
    187.53314700070217,
    112.54862853914825
  ]
}


In [72]:
%%bash

curl -s -X POST "${MODEL_SERVER_URL}/v2/repository/index" \
  -H "Content-Type: application/json" \
  -d "{}" | jq


[
  {
    "name": "diabetes-model",
    "state": "READY",
    "reason": ""
  }
]
